In [ ]:
import numpy as np
import pandas as pd

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
def load_data(path):
    data = pd.read_csv(path)
    return data

In [ ]:
df=load_data("/content/drive/MyDrive/datathon/train.csv")
df.head()

,index,Cinsiyet,Yaş Grubu,Medeni Durum,Eğitim Düzeyi,İstihdam Durumu,Yıllık Ortalama Gelir,Yaşadığı Şehir,En Çok İlgilendiği Ürün Grubu,Yıllık Ortalama Satın Alım Miktarı,Yıllık Ortalama Sipariş Verilen Ürün Adedi,Eğitime Devam Etme Durumu,Öbek İsmi,Yıllık Ortalama Sepete Atılan Ürün Adedi
0,0,Kadın,31-40,Bekar,Yüksek Lisans Mezunu,Düzenli ve Ücretli Bir İşi Var,748266.440879,Büyük Şehir,Elektronik ve Teknolojik Ürünler,32010.900881,37.310644,Etmiyor,obek_4,102.936904
1,1,Erkek,>60,Evli,Lise Mezunu,Kendi İşinin Sahibi,246298.609605,Küçük Şehir,Spor Malzemeleri,4145.729891,11.554851,Etmiyor,obek_5,82.176053
2,2,Erkek,18-30,Bekar,Lise Mezunu,Düzenli ve Ücretli Bir İşi Var,268582.732360,Küçük Şehir,Giyim,9924.379414,55.353236,Ediyor,obek_8,141.661426
3,3,Erkek,51-60,Evli,Lise Mezunu,Kendi İşinin Sahibi,327721.837865,Kırsal,Giyim,6417.780714,15.417239,Etmiyor,obek_3,17.097397
4,4,Erkek,31-40,Evli,Ortaokul Mezunu,İşsiz veya Düzenli Bir İşi Yok,397431.626948,Büyük Şehir,Giyim,7886.663061,14.689837,Etmiyor,obek_3,20.470459


In [ ]:
def outlier_thresholds(dataframe, col_name):
    q1=0.15
    q3=0.65
    quartile1 = dataframe[col_name].quantile(q1)
    quartile3 = dataframe[col_name].quantile(q3)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

In [ ]:
def check_outlier(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
        return col_name
    else:
        return False

In [ ]:
def grab_col_names(dataframe, cat_th=2, car_th=10):
    """

    Veri setindeki kategorik, numerik ve kategorik fakat kardinal değişkenlerin isimlerini verir.
    Not: Kategorik değişkenlerin içerisine numerik görünümlü kategorik değişkenler de dahildir.

    Parameters
    ------
        dataframe: dataframe
                Değişken isimleri alınmak istenilen dataframe
        cat_th: int, optional
                numerik fakat kategorik olan değişkenler için sınıf eşik değeri
        car_th: int, optinal
                kategorik fakat kardinal değişkenler için sınıf eşik değeri

    Returns
    ------
        cat_cols: list
                Kategorik değişken listesi
        num_cols: list
                Numerik değişken listesi
        cat_but_car: list
                Kategorik görünümlü kardinal değişken listesi

    Examples
    ------
        import seaborn as sns
        df = sns.load_dataset("iris")
        print(grab_col_names(df))


    Notes
    ------
        cat_cols + num_cols + cat_but_car = toplam değişken sayısı
        num_but_cat cat_cols'un içerisinde.
        Return olan 3 liste toplamı toplam değişken sayısına eşittir: cat_cols + num_cols + cat_but_car = değişken sayısı

    """

    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')
    return cat_cols, num_cols, cat_but_car

cat_cols, num_cols, cat_but_car = grab_col_names(df)

Observations: 5460
Variables: 14
cat_cols: 9
num_cols: 5
cat_but_car: 0
num_but_cat: 0


In [ ]:
for col in num_cols:
    print(check_outlier(df,col))

False
Yıllık Ortalama Gelir
False
False
Yıllık Ortalama Sepete Atılan Ürün Adedi


In [ ]:
def grab_outliers(dataframe, col_name, index=False):
    low, up = outlier_thresholds(dataframe, col_name)

    if dataframe[((dataframe[col_name] < low) | (dataframe[col_name] > up))].shape[0] > 10:
        print(dataframe[((dataframe[col_name] < low) | (dataframe[col_name] > up))].head())
    else:
        print(dataframe[((dataframe[col_name] < low) | (dataframe[col_name] > up))])

    if index:
        outlier_index = dataframe[((dataframe[col_name] < low) | (dataframe[col_name] > up))].index
        return outlier_index

In [ ]:
print(grab_outliers(df,"Yıllık Ortalama Gelir",index=True))

    index Cinsiyet Yaş Grubu Medeni Durum         Eğitim Düzeyi  \
37     37    Erkek     31-40        Bekar     Yüksekokul Mezunu   
44     44    Erkek     31-40        Bekar  Yüksek Lisans Mezunu   
66     66    Erkek     18-30        Bekar  Yüksek Lisans Mezunu   
68     68    Kadın     18-30        Bekar     Üniversite Mezunu   
84     84    Kadın     31-40        Bekar         Doktora Ötesi   

                   İstihdam Durumu  Yıllık Ortalama Gelir Yaşadığı Şehir  \
37  İşsiz veya Düzenli Bir İşi Yok           9.691073e+05    Küçük Şehir   
44  Düzenli ve Ücretli Bir İşi Var           1.062555e+06    Büyük Şehir   
66  İşsiz veya Düzenli Bir İşi Yok           8.730016e+05    Büyük Şehir   
68  Düzenli ve Ücretli Bir İşi Var           1.039148e+06    Büyük Şehir   
84  Düzenli ve Ücretli Bir İşi Var           1.008028e+06    Büyük Şehir   

       En Çok İlgilendiği Ürün Grubu  Yıllık Ortalama Satın Alım Miktarı  \
37  Elektronik ve Teknolojik Ürünler                        3281

In [ ]:
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [ ]:
df.shape

for col in num_cols:
    replace_with_thresholds(df, col)


In [ ]:
print(df['Yıllık Ortalama Gelir'])

0       748266.440879
1       246298.609605
2       268582.732360
3       327721.837865
4       397431.626948
5       332489.316371
6       256538.898810
7       213630.355636
8       479593.473447
9       477359.775587
10      213570.560610
11      312415.375463
12      344598.478950
13      217216.075154
14      361022.430491
15      234827.322134
16      332390.941182
17      488521.817097
18      505257.448313
19      493250.254095
20      238806.105931
21      229107.734142
22      269360.841748
23      305712.117451
24      141995.912615
25      237309.658499
26      235677.031889
27      309105.538569
28      264184.814084
29      484971.258930
30      358613.754008
31      296415.326819
32      231552.546236
33      197818.780780
34      355795.176305
35      180379.993977
36      493873.029253
37      846930.395534
38      296007.966906
39      227718.685204
40      327841.377807
41      226346.988886
42      475062.624202
43      225775.182647
44      846930.395534
45      47

In [ ]:
df.apply(lambda x: x.fillna(x.median()) if x.dtype != "O" else x, axis=0).isnull().sum()
# kdf=ategorik değişkenleri mode ile doldurma
df.apply(lambda x: x.fillna(x.mode()[0]) if (x.dtype == "O" and len(x.unique()) <= 10) else x, axis=0).isnull().sum()

index                                         0
Cinsiyet                                      0
Yaş Grubu                                     0
Medeni Durum                                  0
Eğitim Düzeyi                                 0
İstihdam Durumu                               0
Yıllık Ortalama Gelir                         0
Yaşadığı Şehir                                0
En Çok İlgilendiği Ürün Grubu                 0
Yıllık Ortalama Satın Alım Miktarı            0
Yıllık Ortalama Sipariş Verilen Ürün Adedi    0
Eğitime Devam Etme Durumu                     0
Öbek İsmi                                     0
Yıllık Ortalama Sepete Atılan Ürün Adedi      0
dtype: int64

In [ ]:
print(df.head())

   index Cinsiyet Yaş Grubu Medeni Durum         Eğitim Düzeyi  \
0    0.0    Kadın     31-40        Bekar  Yüksek Lisans Mezunu   
1    1.0    Erkek       >60         Evli           Lise Mezunu   
2    2.0    Erkek     18-30        Bekar           Lise Mezunu   
3    3.0    Erkek     51-60         Evli           Lise Mezunu   
4    4.0    Erkek     31-40         Evli       Ortaokul Mezunu   

                  İstihdam Durumu  Yıllık Ortalama Gelir Yaşadığı Şehir  \
0  Düzenli ve Ücretli Bir İşi Var          748266.440879    Büyük Şehir   
1             Kendi İşinin Sahibi          246298.609605    Küçük Şehir   
2  Düzenli ve Ücretli Bir İşi Var          268582.732360    Küçük Şehir   
3             Kendi İşinin Sahibi          327721.837865         Kırsal   
4  İşsiz veya Düzenli Bir İşi Yok          397431.626948    Büyük Şehir   

      En Çok İlgilendiği Ürün Grubu  Yıllık Ortalama Satın Alım Miktarı  \
0  Elektronik ve Teknolojik Ürünler                        32010.900881   
1 

In [ ]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder , RobustScaler
def label_encoder(dataframe, cols):
    labelencoder = LabelEncoder()
    for binary_col in cols:
      dataframe[binary_col] = labelencoder.fit_transform(dataframe[binary_col])
    return dataframe

In [ ]:
new_df=label_encoder(df,cat_cols)
print(new_df.head())

   index  Cinsiyet  Yaş Grubu  Medeni Durum  Eğitim Düzeyi  İstihdam Durumu  \
0    0.0         1          1             0              5                0   
1    1.0         0          4             1              3                2   
2    2.0         0          0             0              3                0   
3    3.0         0          3             1              3                2   
4    4.0         0          1             1              4                3   

   Yıllık Ortalama Gelir  Yaşadığı Şehir  En Çok İlgilendiği Ürün Grubu  \
0          748266.440879               0                              0   
1          246298.609605               2                              4   
2          268582.732360               2                              2   
3          327721.837865               3                              2   
4          397431.626948               0                              2   

   Yıllık Ortalama Satın Alım Miktarı  \
0                        32010.90

In [ ]:
def robust_scaler(df, columns, new_df):
    scaler = RobustScaler()
    for col in columns:
        scaled_col = scaler.fit_transform(df[[col]])
        new_df[col] = scaled_col

    return new_df

In [ ]:

new_df=robust_scaler(df,num_cols,new_df)

In [ ]:
print(new_df.tail())
csv_filename = 'purelyData.csv'  # Kaydetmek istediğiniz dosya adını burada belirtin
new_df.to_csv(csv_filename, index=True)

         index  Cinsiyet  Yaş Grubu  Medeni Durum  Eğitim Düzeyi  \
5455  0.998535         1          4             0              7   
5456  0.998901         0          3             1              1   
5457  0.999267         1          3             1              8   
5458  0.999634         1          0             0              6   
5459  1.000000         0          0             0              6   

      İstihdam Durumu  Yıllık Ortalama Gelir  Yaşadığı Şehir  \
5455                1               0.756961               1   
5456                2               0.772006               1   
5457                0               0.089183               2   
5458                0              -0.080482               0   
5459                0              -0.042656               0   

      En Çok İlgilendiği Ürün Grubu  Yıllık Ortalama Satın Alım Miktarı  \
5455                              3                            1.355128   
5456                              1                     

In [ ]:
print(new_df.head())

      index  Cinsiyet  Yaş Grubu  Medeni Durum  Eğitim Düzeyi  \
0 -1.000000         1          1             0              5   
1 -0.999634         0          4             1              3   
2 -0.999267         0          0             0              3   
3 -0.998901         0          3             1              3   
4 -0.998535         0          1             1              4   

   İstihdam Durumu  Yıllık Ortalama Gelir  Yaşadığı Şehir  \
0                0               1.827131               0   
1                2              -0.160902               2   
2                0              -0.072646               2   
3                2               0.161573               3   
4                3               0.437657               0   

   En Çok İlgilendiği Ürün Grubu  Yıllık Ortalama Satın Alım Miktarı  \
0                              0                            0.919551   
1                              4                           -0.166921   
2                         

In [ ]:
y=new_df["Öbek İsmi"]
X=new_df.drop(["Öbek İsmi"],axis=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_validate
from xgboost import XGBClassifier


In [ ]:
def models(X,y, scoring="roc_auc"):
  classifiers=[("KNN", KNeighborsClassifier()),
               ("CART", DecisionTreeClassifier()),
               ("RF", RandomForestClassifier()),
               ("Adaboost", AdaBoostClassifier()),
                ("GBM", GradientBoostingClassifier()),
               ("XGBoost",XGBClassifier(use_label_encoder=False ,eval_metric='logloss')),
               ("LightGBM", LGBMClassifier())]
  for name, classifier in classifiers:
        cv_results = cross_validate(classifier, X, y, cv=10, scoring=scoring)
        print(f"{scoring}: {round(cv_results['test_score'].mean(), 4)} ({name}) ")


print(models(X,y,scoring="accuracy"))


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


accuracy: 0.8687 (KNN) 
accuracy: 0.8601 (CART) 
accuracy: 0.9548 (RF) 
accuracy: 0.8619 (Adaboost) 
accuracy: 0.9256 (GBM) 


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarnin

accuracy: 0.9526 (XGBoost) 
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000337 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1308
[LightGBM] [Info] Number of data points in the train set: 4914, number of used features: 13
[LightGBM] [Info] Start training from score -2.065297
[LightGBM] [Info] Start training from score -2.299334
[LightGBM] [Info] Start training from score -2.060493
[LightGBM] [Info] Start training from score -2.025953
[LightGBM] [Info] Start training from score -2.046219
[LightGBM] [Info] Start training from score -2.071738
[LightGBM] [Info] Start training from score -2.062092
[LightGBM] [Info] Start training from score -2.030593
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto

In [ ]:
knn_params = {"n_neighbors": range(2, 50)}

cart_params = {'max_depth': range(1, 20),
               "min_samples_split": range(2, 30)}

rf_params = {"max_depth": [8, 25, None],
             "max_features": [5, 7, "auto"],
             "min_samples_split": [15, 40],
             "n_estimators": [200, 500]}

xgboost_params = {"learning_rate": [0.1, 0.01],
                  "max_depth": [5, 8],
                  "n_estimators": [100, 200]}

lightgbm_params = {"learning_rate": [0.001, 0.1],
                   "n_estimators": [300, 500]}

classifiers = [("RF", RandomForestClassifier(), rf_params),
               ('XGBoost', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), xgboost_params),
               ('LightGBM', LGBMClassifier(), lightgbm_params)]

NameError: ignored

In [ ]:

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, cross_val_score


def hyperparameter_optimization(X, y, classifiers, cv=10, scoring="roc_auc"):
  top_models = []  # En iyi modelleri saklamak için boş bir liste

  for name, classifier, params in classifiers:
    print(f"Optimizing {name}...")

    # Modeli eğitim verileriyle kurma
    final_model = classifier

    # Eğitim ve test verilerini bölelim


    # Modelin hiperparametrelerini optimize etmek için çapraz doğrulama yapalım
    cv_results = cross_val_score(final_model, X, y,cv=cv, scoring=scoring)

    # Modelleri performanslarına göre sıralayıp en iyi modelleri saklayalım
    top_models.append((name, final_model, cv_results.mean()))

  # Modelleri performanslarına göre sıralayalım
  top_models.sort(key=lambda x: x[2], reverse=True)

  return top_models



top_models = hyperparameter_optimization(X, y, classifiers, scoring="accuracy")

# En iyi 3 modeli yazdır
print("Top 3 Models:")
for idx, (name, model, score) in enumerate(top_models[:3]):
    print(f"{idx + 1}. {name}: {score:.4f}")

Optimizing RF...
Optimizing XGBoost...


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarnin

Optimizing LightGBM...
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000204 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1308
[LightGBM] [Info] Number of data points in the train set: 4914, number of used features: 13
[LightGBM] [Info] Start training from score -2.065297
[LightGBM] [Info] Start training from score -2.299334
[LightGBM] [Info] Start training from score -2.060493
[LightGBM] [Info] Start training from score -2.025953
[LightGBM] [Info] Start training from score -2.046219
[LightGBM] [Info] Start training from score -2.071738
[LightGBM] [Info] Start training from score -2.062092
[LightGBM] [Info] Start training from score -2.030593
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choo

In [ ]:


def predict(top_models, path , X ,y):


  test_data=load_data(path)
  cat_cols, num_cols, cat_but_car = grab_col_names(test_data)
  for col in num_cols:
    replace_with_thresholds(test_data, col)


  test_data.apply(lambda x: x.fillna(x.median()) if x.dtype != "O" else x, axis=0).isnull().sum()
# kdf=ategorik değişkenleri mode ile doldurma
  test_data.apply(lambda x: x.fillna(x.mode()[0]) if (x.dtype == "O" and len(x.unique()) <= 10) else x, axis=0).isnull().sum()
  new_test_df=label_encoder(test_data,cat_cols)
  new_test_df=robust_scaler(test_data,num_cols,new_test_df)
  csv_filename = 'test.csv'  # Kaydetmek istediğiniz dosya adını burada belirtin
  new_test_df.to_csv(csv_filename, index=True)

  predictions_df = pd.DataFrame(index=test_data.index)

  RF_model=top_models[0][1]
  RF_model.fit(X,y)

  predictions=RF_model.predict(new_test_df)



  predictions_df = pd.DataFrame(predictions + 1, columns=["Öbek İsmi"], index=new_test_df.index)
  predictions_df["Öbek İsmi"] = "obek_"+predictions_df["Öbek İsmi"].astype(str)
  predictions_df.index.name = "id"

  return predictions_df




In [ ]:
predictions_df= predict(top_models,"/content/drive/MyDrive/datathon/test_x.csv",X,y)


Observations: 2340
Variables: 13
cat_cols: 8
num_cols: 5
cat_but_car: 0
num_but_cat: 0


In [ ]:
csv_filename = 'predictions.csv'  # Kaydetmek istediğiniz dosya adını burada belirtin
predictions_df.to_csv(csv_filename, index=True)

NameError: ignored